#### Imports and Strategy Shorthands

3/20/20: So far I have set up this code in order to create the full circuit on an arbitrary set of initial particles for an arbitrary number of steps. It remains to be tested and simulated 

In [28]:
#requires cirq version >= v0.6.0
import cirq
import math
import numpy as np
from cirq import GridQubit, X, CNOT, TOFFOLI, H, Ry
early = cirq.InsertStrategy.EARLIEST
new = cirq.InsertStrategy.NEW_THEN_INLINE

#### Classically pre-calculate parameter lists

In [29]:
#Define splitting functions and sudakov factors
def P_f(t, g):
    alpha = g**2 * Phat_f(t)/ (4 * math.pi)
    return alpha
def Phat_f(t):
    return(math.log(t))
def Phat_bos(t):
    return(math.log(t))
def Delta_f(t, g):
    return math.exp(P_f(t,g))
def P_bos(t, g_a, g_b):
    alpha = g_a**2 * Phat_bos(t)/ (4 * math.pi) + g_b**2 * Phat_bos(t)/ (4 * math.pi)
    return alpha
def Delta_bos(t, g_a, g_b):
    return math.exp(P_bos(t, g_a, g_b))

In [30]:
#Populates the 7 lists below with correct values for each time step theta
def populateParameterLists(N, timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, Delta_phiList, g_a, g_b):
    for i in range(N):
       # Compute time steps
        t_up = eps**((i)/N)
        t_mid =  eps**((i+0.5)/N)
        t_low =  eps**((i+1)/N)
        timeStepList.append(t_mid)

       # Compute values for emission matrices
        Delta_a = math.sqrt(Delta_f(t_low, g_a)) / math.sqrt(Delta_f(t_up, g_a))
        Delta_b = math.sqrt(Delta_f(t_low, g_b)) / math.sqrt(Delta_f(t_up, g_b))   
        Delta_phi = math.sqrt(Delta_bos(t_low, g_a, g_b)) / math.sqrt(Delta_bos(t_up, g_a, g_b))
        P_a, P_b, P_phi = P_f(t_mid, g_a), P_f(t_mid, g_b), P_bos(t_mid, g_a, g_b)
                              
       # Add them to the list
        P_aList.append(P_a)
        P_bList.append(P_b)
        P_phiList.append(P_phi)
        Delta_aList.append(Delta_a)
        Delta_bList.append(Delta_b)
        Delta_phiList.append(Delta_phi)

#### Allocate qubits on Hardware 

In [31]:
#assign Qubits (can change based on hardware implementation!)
def allocateQubs(N, n_i, L, pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg):
    pReg.extend([[GridQubit(i,j) for j in range(4)] for i in range(N+n_i)]) 
    hReg.extend([[GridQubit(i,j) for j in range(4, 4+L)] for i in range(N)])
    w_hReg.extend([GridQubit(N+n_i,j) for j in range(L-1)]) 
    eReg.extend([GridQubit(N+n_i,L-1)])
    wReg.extend([GridQubit(N+n_i,j) for j in range(L,L+5)])
    n_phiReg.extend([GridQubit(N+n_i+1,j) for j in range(L)])
    w_phiReg.extend([GridQubit(N+n_i+1,j) for j in range(L,2*L-1)])
    n_aReg.extend([GridQubit(N+n_i+2,j) for j in range(L)])
    w_aReg.extend([GridQubit(N+n_i+2,j) for j in range(L,2*L-1)])
    n_bReg.extend([GridQubit(N+n_i+3,j) for j in range(L)])
    w_bReg.extend([GridQubit(N+n_i+3,j) for j in range(L,2*L-1)])

#apply appropriate X gates to ensure that the P register contains all of the initial particles (passed in as a list)    
def intializeParticles(circuit, pReg, initialParticles):
    currentParticleIndex = 0
    for particle in initialParticles:
        for particleBit in range(3): #fourth bit is work register - always starts at 0
            if initialParticles[currentParticleIndex][particleBit] == 1:
                circuit.append(X(pReg[currentParticleIndex][particleBit]), strategy=early)
        currentParticleIndex += 1

#### U_Count Helper Functions

In [32]:
#controlled x onto targetQubit if control is of the correct flavor
def flavorControl(circuit, flavor, control, target):
    if flavor == "phi":
        circuit.append([X(control[1]), X(control[2])], strategy = new)
        circuit.append(TOFFOLI(control[0], control[1], control[3]), strategy = new)
        circuit.append(TOFFOLI(control[2], control[3], target), strategy = new)
        circuit.append([X(control[1]), X(control[2])], strategy = new)
    if flavor == "a":
        circuit.append(X(control[0]), strategy = new)
        circuit.append(TOFFOLI(control[0], control[2], target), strategy = new)
        circuit.append(X(control[0]), strategy = new)
    if flavor == "b":
        circuit.append(TOFFOLI(control[0], control[2], target), strategy = new)

#recursively carries an addition of 1 to the LSB of a register to all bits (if neccesarry)
def plus1(circuit, l, countReg, workReg, level):
    if level <= l-2:
        #first level uses CNOT instead of TOFFOLI gate
        if level==0:
            #move all X gates to first step to avoid unnecesarry gates
            circuit.append([X(qubit) for qubit in countReg], strategy=new)
            circuit.append(CNOT(countReg[0], workReg[0]), strategy=new)      
        else:
            circuit.append(TOFFOLI(countReg[level], workReg[level-1], workReg[level]), strategy=new)
           
        circuit.append(CNOT(workReg[level], countReg[level+1]), strategy=new)
        #recursively call next layer
        plus1(circuit, l, countReg, workReg, level+1)
        #undo work qubits (exact opposite of first 5 lines - undoes calculation)
        if level==0:
            circuit.append(CNOT(countReg[0], workReg[0]), strategy=new) 
            circuit.append([X(qubit) for qubit in countReg], strategy=new)              
        else:
            circuit.append(TOFFOLI(countReg[level], workReg[level-1], workReg[level]), strategy=new)
        


### U_Count

In [33]:
#populate the count registers using current particle states
def uCount(circuit, m, n_i, l, pReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg):
    for k in range(n_i+m):
        flavorControl(circuit, "phi", pReg[k], n_phiReg[0])
        flavorControl(circuit, "a", pReg[k], n_aReg[0])
        flavorControl(circuit, "b", pReg[k], n_bReg[0])
        plus1(circuit, l, n_phiReg, w_phiReg, 0)
        plus1(circuit, l, n_aReg, w_aReg, 0)
        plus1(circuit, l, n_aReg, w_bReg, 0)

#### Utility Functions

In [34]:
#fill countsList with all compinations of n_phi, n_a, and n_b from [0,n+m] that satisfy a sum from [n, n+m] 
def generateParticleCounts(n_i, m, countsList):
    minParticles = n_i
    maxParticles = n_i+m
    for numParticles in range(minParticles, maxParticles+1):
        for numPhi in range(0, maxParticles+1):
            for numA in range(0, numParticles - numPhi + 1):
                numB = maxParticles - numPhi - numA
                countsList.append([numPhi, numA, numB])
    #print(countsList)
    return

def reverse(lst): 
    lst.reverse() 
    return lst 

#converts integer to binary list of size l with LSB first and MSB last
def intToBinary(l, number):
    numberBinary = [int(x) for x in list('{0:0b}'.format(number))]
    numberBinary = (l - len(numberBinary)) * [0] + numberBinary
    return reverse(numberBinary) 

In [35]:
#applies an X to the target register if countRegister encodes the inputted number in binary, return target
#DOES NOT CLEAN AFTER ITSELF - USE numberControlT
def numberControl(circuit, l, number, countReg, workReg):
    if type(number)==int:
        numberBinary = intToBinary(l, number)
    else:
        numberBinary = number
    circuit.append([X(countReg[i]) for i in range(len(numberBinary)) if numberBinary[i]==0], strategy=new)
    #first level does not use work qubits as control
    if l > 1:
        circuit.append(TOFFOLI(countReg[0], countReg[1], workReg[0]), strategy=new)  
    #subfunction to recursively handle toffoli gates
    def binaryToffolis(level):
        circuit.append(TOFFOLI(countReg[level], workReg[level-2], workReg[level-1]), strategy=new)
        if level <= l - 2:
            binaryToffolis(level+1)    
    if l > 2:
        binaryToffolis(2)
    #return qubit containing outcome of the operation
    if l == 1:
        return countReg[0]
    else:
        return workReg[l-2]
    
#undoes numberControl operation. CLEANS AFTER numberControl
def numberControlT(circuit, l, number, countReg, workReg):
    if type(number)==int:
        numberBinary = intToBinary(l, number)    
    else:
        numberBinary = number
    #subfunction to recursively handle toffoli gates
    def binaryToffolisT(level):
        #circuit.append(TOFFOLI(countReg[level], workReg[level-2], workReg[level-1]), strategy=new)
        if level <= l - 1:
            binaryToffolisT(level+1)
            #undo
            circuit.append(TOFFOLI(countReg[level], workReg[level-2], workReg[level-1]), strategy=new)            
    if l > 2:
        binaryToffolisT(2)
        #undo
    if l > 1:
        circuit.append(TOFFOLI(countReg[0], countReg[1], workReg[0]), strategy=new)   
    #undo        
    circuit.append([X(countReg[i]) for i in range(len(numberBinary)) if numberBinary[i]==0], strategy=new)

### U_e 

In [36]:
def uE(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg, Delta_phi, Delta_a, Delta_b):
    countsList = []
    generateParticleCounts(n_i, m, countsList)
    for counts in countsList:
        n_phi, n_a, n_b = counts[0], counts[1], counts[2]
        Delta = Delta_phi**n_phi * Delta_a**n_a * Delta_b**n_b
        phiControlQub = numberControl(circuit, l, n_phi, n_phiReg, w_phiReg)
        aControlQub = numberControl(circuit, l, n_a, n_aReg, w_aReg)
        bControlQub = numberControl(circuit, l, n_b, n_bReg, w_bReg)
        circuit.append(TOFFOLI(phiControlQub, aControlQub, wReg[0]), strategy=new)
        circuit.append(TOFFOLI(bControlQub, wReg[0], wReg[1]), strategy=new) 
        circuit.append(Ry(2*math.acos(np.sqrt(Delta))/np.pi).controlled().on(wReg[1], eReg[0]))
        #undo
        circuit.append(TOFFOLI(bControlQub, wReg[0], wReg[1]), strategy=new)
        circuit.append(TOFFOLI(phiControlQub, aControlQub, wReg[0]), strategy=new)
        numberControlT(circuit, l, n_b, n_bReg, w_bReg)   
        numberControlT(circuit, l, n_a, n_aReg, w_aReg)
        numberControlT(circuit, l, n_phi, n_phiReg, w_phiReg) 

 #### U_h helper functions (Arbitary 2 level controlled unitary, angle generator, minus 1 gate)

In [37]:
#return list of elements in gray code from |0> to |number> where each entry is of type[int, binary list].
#int: which bit is the target in the current iteration, binary list: the state of the rest of the qubits (controls) 
def generateGrayList(l,number):
    grayList = [[0,l * [0]]]
    targetBinary = intToBinary(l, number)
    for index in range(len(targetBinary)):
        if targetBinary[index] == 1:
            grayList.append([index,(list(grayList[-1][1]))])
            grayList[-1][1][index] = 1
    return grayList[1:]

#implement two level Ry rotation from state |0> to |k>, if externalControl qubit is on
#for reference: http://www.physics.udel.edu/~msafrono/650/Lecture%206.pdf
def twoLevelControlledRy(circuit, l, angle, k, externalControl, reg, workReg):
    grayList = generateGrayList(l, k)
    #handle the case where l=1
    if l==1 and k == 1:
        circuit.append(cirq.Ry(angle).controlled().on(externalControl, reg[0]))
        return
             
    #swap states according to Gray Code until one step before the end
    for element in grayList:
        targetQub = element[0]
        number = element[1]
        number = number[0:targetQub] + number[targetQub+1:]
        controlQub = numberControl(circuit, l-1, number, reg[0:targetQub]+reg[targetQub+1:], workReg)
        if element == grayList[-1]: #reached end
            circuit.append(TOFFOLI(controlQub, externalControl, workReg[l-2]), strategy = new)
            circuit.append(cirq.Ry(angle).controlled().on(workReg[l-2], reg[targetQub]))
            circuit.append(TOFFOLI(controlQub, externalControl, workReg[l-2]), strategy = new)            
        else: #swap states
            circuit.append(CNOT(controlQub, reg[targetQub]), strategy = new)
        numberControlT(circuit, l-1, number, reg[0:targetQub]+reg[targetQub+1:], workReg)
    
    #undo
    for element in reverse(grayList[:-1]):
        targetQub = element[0]
        number = element[1]
        number = number[0:targetQub] + number[targetQub+1:]
        controlQub = numberControl(circuit, l-1, number, reg[0:targetQub]+reg[targetQub+1:], workReg)
        circuit.append(CNOT(controlQub, reg[targetQub]), strategy = new)
        numberControlT(circuit, l-1, number, reg[0:targetQub]+reg[targetQub+1:], workReg)   
    return

In [38]:
def U_hAngle(flavor, n_phi, n_a, n_b, P_phi, P_a, P_b):
    denominator = n_phi * P_phi + n_a * P_a + n_b * P_b
    flavorStringToP = {'phi':P_phi, 'a':P_a, 'b':P_b}
    emissionAmplitude = np.sqrt(flavorStringToP[flavor] / denominator)
    #correct for arcsin input greater than 1 errors for various input combinations that are irrelevant anyway
    emissionAmplitude = min(1, emissionAmplitude)
    return (2 * np.arcsin(emissionAmplitude)) / np.pi

#recursively carries an subtraction of 1 to the LSB of a register to all bits (if neccesarry)
#Equivalent to plus1 but with an X applied to all count qubits before and after gate
def minus1(circuit, l, countReg, workReg, level):
    circuit.append([X(qubit) for qubit in countReg], strategy=new)
    plus1(circuit, l, countReg, workReg, level)
    circuit.append([X(qubit) for qubit in countReg], strategy=new)

### U_h

In [39]:
#implement U_h on all registers
#TODO: MAKE SURE THIS MAKES SENSE ON HIGH LEVEL
def U_h(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg, pReg, hReg, w_hReg, P_phi, P_a, P_b):
    countsList = []
    generateParticleCounts(n_i, m, countsList)
    for k in range(n_i + m):
        for counts in countsList:
            n_phi, n_a, n_b = counts[0], counts[1], counts[2]
            #controlled R-y from |0> to |k> on all qubits with all possible angles depending on n_phi, n_a, n_b, and flavor
            for flavor in ['phi', 'a', 'b']:
                angle = U_hAngle(flavor, n_phi, n_a, n_b, P_phi, P_a, P_b)
                phiControl = numberControl(circuit, l, n_phi, n_phiReg, w_phiReg)
                aControl = numberControl(circuit, l, n_a, n_aReg, w_aReg)
                bControl = numberControl(circuit, l, n_b, n_bReg, w_bReg)
                circuit.append(TOFFOLI(phiControl, aControl, wReg[0]), strategy=new)
                circuit.append(TOFFOLI(bControl, wReg[0], wReg[1]), strategy=new) 
                flavorControl(circuit, flavor, pReg[k], wReg[2])
                circuit.append(TOFFOLI(eReg[0], wReg[2], wReg[3]), strategy=new)
                twoLevelControlledRy(circuit, l, angle, k, wReg[3], hReg[m], w_hReg)
                #undo
                circuit.append(TOFFOLI(eReg[0], wReg[2], wReg[3]), strategy=new)
                circuit.append(TOFFOLI(bControl, wReg[0], wReg[1]), strategy=new)
                circuit.append(TOFFOLI(phiControl, aControl, wReg[0]), strategy=new)
                numberControlT(circuit, l, n_b, n_bReg, w_bReg)
                numberControlT(circuit, l, n_a, n_aReg, w_aReg)
                numberControlT(circuit, l, n_phi, n_phiReg, w_phiReg)
        
        #subtract from the counts register depending on which flavor particle emitted
        for flavor, countReg, workReg in zip(['phi','a','b'],[n_phiReg, n_aReg, n_bReg],[w_phiReg, w_aReg, w_bReg]):
            flavorControl(circuit, flavor, pReg[k], countReg[0])
            minus1(circuit, l, countReg, workReg, 0)
            flavorControl(circuit, flavor, pReg[k], countReg[0])

    #apply not 0 controlled x from the mth h register to the e register
    numberControl(circuit, l, 0, hReg[m], w_hReg)
    circuit.append(CNOT(w_hReg[l-2], eReg[0]))
    circuit.append(X(eReg[0]), strategy=new)
    numberControlT(circuit, l, 0, hReg[m], w_hReg)
                       

#### U_p Helper Function and U_p

In [40]:
#updates old particle k and new particle n_i+m (0-indexed) if particle k undergoes emission in step n+m
def updateParticles(circuit, l, n_i, m, k, pReg, wReg, controlQub, g_a, g_b):
    oldParticleReg = pReg[k]
    newParticleReg = pReg[n_i+m]
    #first gate in paper U_p
    circuit.append(TOFFOLI(controlQub, oldParticleReg[2], newParticleReg[0]), strategy=new)
    #second gate in paper (undoes work register immediately)
    circuit.append([X(oldParticleReg[1]), X(oldParticleReg[2])], strategy=new)
    circuit.append(TOFFOLI(controlQub, oldParticleReg[2], wReg[0]), strategy=new)
    circuit.append(TOFFOLI(wReg[0], oldParticleReg[1], wReg[1]), strategy=new)
    circuit.append(TOFFOLI(wReg[1], oldParticleReg[0], newParticleReg[2]), strategy=new)
    circuit.append(TOFFOLI(wReg[0], oldParticleReg[1], wReg[1]), strategy=new)
    circuit.append(TOFFOLI(controlQub, oldParticleReg[2], wReg[0]), strategy=new)
    circuit.append([X(oldParticleReg[1]), X(oldParticleReg[2])], strategy=new)
    #third gate in paper
    circuit.append(TOFFOLI(controlQub, newParticleReg[2], oldParticleReg[2]), strategy=new)
    #fourth and fifth gate in paper (then undoes work register)
    circuit.append(TOFFOLI(controlQub, newParticleReg[2], wReg[0]), strategy=new)
    circuit.append(cirq.H.controlled().on(wReg[0], newParticleReg[1]))
    angle = (2 * np.arccos(g_a/np.sqrt(g_a**2 + g_b**2))) / np.pi
    circuit.append(cirq.Ry(angle).controlled().on(wReg[0], newParticleReg[0]))
    circuit.append(TOFFOLI(controlQub, newParticleReg[2], wReg[0]), strategy=new)
    #sixth and seventh gate in paper (then undoes work register)
    circuit.append([X(newParticleReg[0]), X(newParticleReg[1])], strategy=new)
    circuit.append(TOFFOLI(newParticleReg[1], newParticleReg[2], wReg[0]), strategy=new)
    circuit.append(TOFFOLI(controlQub, wReg[0], oldParticleReg[1]), strategy=new)
    circuit.append(TOFFOLI(newParticleReg[1], newParticleReg[2], wReg[0]), strategy=new)
    circuit.append(TOFFOLI(newParticleReg[0], newParticleReg[2], wReg[0]), strategy=new)
    circuit.append(TOFFOLI(controlQub, wReg[0], oldParticleReg[0]), strategy=new)
    circuit.append(TOFFOLI(newParticleReg[0], newParticleReg[2], wReg[0]), strategy=new)
    circuit.append([X(newParticleReg[0]), X(newParticleReg[1])], strategy=new)
    
    
    

In [41]:
#Apply U_p to circuit
def U_p(circuit, l, n_i, m, pReg, hReg, w_hReg, wReg, g_a, g_b):
    for k in range(0, n_i + m):
        controlQub = numberControl(circuit, l, k, hReg[m], w_hReg)
        updateParticles(circuit, l, n_i, m, k, pReg, wReg, controlQub, g_a, g_b)
        numberControlT(circuit, l, k, hReg[m], w_hReg)

## Create Circuit

In [42]:
def createCircuit(n_i, N, eps, g_1, g_2, g_12, initialParticles):
    #calculate constants
    gp = ((g_2-g_1) / abs(g_2-g_1)) * math.sqrt((g_1-g_2)**2 + 4*g_12**2)
    g_a, g_b = (g_1 + g_2 - gp) / 2, (g_1 + g_2 + gp) / 2
    u = math.sqrt((g_1-g_2+gp)/2*gp)
    L = int(math.floor(math.log(N+n_i,2)) + 1)
    
    #evaluate P(Theta) and Delta(Theta) at every time step
    timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, Delta_phiList = [],[],[],[],[],[],[]
    populateParameterLists(N, timeStepList, P_aList, P_bList, P_phiList, Delta_aList, Delta_bList, Delta_phiList, g_a, g_b)
    
    #allocate and populate registers
    pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg = [],[],[],[],[],[],[],[],[],[],[]
    allocateQubs(N, n_i, L, pReg, hReg, w_hReg, eReg, wReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)
    
    #create circuit object and initialize particles
    circuit = cirq.Circuit()
    intializeParticles(circuit, pReg, initialParticles)
    
    #begin stepping through subcircuits
    for m in range(N):
        l = int(math.floor(math.log(m+n_i,2)) + 1)       
        #R^(m) rotate every particle p_k from 1,2 to a,b basis (step 1)
        for p_k in pReg:
            circuit.append(Ry(2*math.asin(-u)/np.pi).controlled().on(p_k[2], p_k[0]))
        #populate count register (step 2)
        uCount(circuit, m, n_i, l, pReg, n_aReg, w_aReg, n_bReg, w_bReg, n_phiReg, w_phiReg)
        #assess if emmision occured (step 3)
        uE(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg, Delta_phiList[m], Delta_aList[m], Delta_bList[m])
        #choose a particle to split (step 4)
        U_h(circuit, l, n_i, m, n_phiReg, w_phiReg, n_aReg, w_aReg, n_bReg, w_bReg, wReg, eReg, pReg, hReg, w_hReg, P_phiList[m], P_aList[m], P_bList[m])
        #update particle based on which particle split/emmitted (step 5)
        U_p(circuit, l, n_i, m, pReg, hReg, w_hReg, wReg, g_a, g_b)
        #R^-(m) rotate every particle p_k from a,b to 1,2 basis (step 6)
        for p_k in pReg:
            circuit.append(Ry(2*math.asin(u)/np.pi).controlled().on(p_k[2], p_k[0]))
    return circuit
    #print(circuit)

#Following paper's notation: n_i -> number of initial particles, N -> number of steps 
g_1, g_2, g_12,  eps = 2, 1, .1, .001
n_i, N = 2, 3
#run circuit with 1 f_a fermion
circuit = createCircuit(n_i, N, eps, g_1, g_2, g_12, [[1,0,0,0]])